In [ ]:
import pandas as pd
import requests

url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": [40.712776, 41.82399, 42.440498, 43.700859, 42.3656347, 40.3496953, 39.9527237, 41.3082138],
    "longitude": [-74.005974, -71.412834, -76.495697, -72.289398, -71.1040018, -74.6597376, -75.1635262, -72.9250518],
    "daily": "temperature_2m_min",
    "hourly": ["temperature_2m", "sunshine_duration"],
    "current": ["precipitation", "snowfall"],
    "timezone": "America/Chicago",
    "wind_speed_unit": "mph",
    "temperature_unit": "fahrenheit",
    "precipitation_unit": "inch",
    "start_date": "2026-01-01",
    "end_date": "2026-01-31",
}

response = requests.get(url, params=params)
data = response.json()

all_hourly = []
all_daily = []

for i, loc in enumerate(data):
    print(f"\nLocation {i+1}: {loc['latitude']}°N, {loc['longitude']}°W")
    print(f"Elevation: {loc['elevation']} m asl")
    print(f"Timezone: {loc['timezone']} ({loc['timezone_abbreviation']})")
    
    current = loc.get('current', {})
    print(f"Current precipitation: {current.get('precipitation')} in")
    print(f"Current snowfall: {current.get('snowfall')} in")
    
    hourly = loc.get('hourly', {})
    hourly_df = pd.DataFrame({
        'location': i+1,
        'latitude': loc['latitude'],
        'longitude': loc['longitude'],
        'time': hourly.get('time', []),
        'temperature_2m': hourly.get('temperature_2m', []),
        'sunshine_duration': hourly.get('sunshine_duration', [])
    })
    all_hourly.append(hourly_df)
    
    daily = loc.get('daily', {})
    daily_df = pd.DataFrame({
        'location': i+1,
        'latitude': loc['latitude'],
        'longitude': loc['longitude'],
        'date': daily.get('time', []),
        'temperature_2m_min': daily.get('temperature_2m_min', [])
    })
    all_daily.append(daily_df)

hourly_data = pd.concat(all_hourly, ignore_index=True)
daily_data = pd.concat(all_daily, ignore_index=True)

print("\n--- Hourly Data Sample ---")
print(hourly_data.head(10))

print("\n--- Daily Data Sample ---")
print(daily_data.head(10))

In [ ]:
IVY_LEAGUES = [
    "Harvard University",
    "Yale University",
    "Princeton University",
    "Columbia University",
    "University of Pennsylvania",
    "Brown University",
    "Dartmouth College",
    "Cornell University"
]

In [ ]:
import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import col

def fetch_ivy_league_data(session: snowpark.Session) -> snowpark.DataFrame:
    import requests
    import pandas as pd
    
    API_KEY = "_________"
    BASE_URL = "https://api.data.gov/ed/collegescorecard/v1/schools"
    
 
    
    fields = [
        "school.name",
        "school.city",
        "school.state",
        "latest.student.size"
    ]
    
    results = []
    
    for school in IVY_LEAGUES:
        params = {
            "api_key": API_KEY,
            "school.name": school,
            "fields": ",".join(fields),
            "per_page": 1
        }
        
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        data = response.json()
        
        if data["results"]:
            record = data["results"][0]
            results.append({
                "SCHOOL": record.get("school", {}).get("name", "N/A"),
                "CITY": record.get("school", {}).get("city", "N/A"),
                "STATE": record.get("school", {}).get("state", "N/A"),
                "ENROLLMENT": record.get("latest", {}).get("student", {}).get("size", None)
            })
    
    # Convert pandas DataFrame to Snowpark DataFrame
    pandas_df = pd.DataFrame(results)
    return session.create_dataframe(pandas_df)